## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Nov. 10th Nov. 17th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,November-15-2024,Condo/Co-op,540 West Ave #1713,Miami Beach,FL,33139.0,793000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/540-West...,MARMLS,A11693360,N,Y,25.77524,-80.141945


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
89,PAST SALE,2024-11-18,Condo/Co-op,10201 Collins Ave #2406,Bal Harbour,FL,33154.0,6562500.0,2.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,MARMLS,A11559220,N,Y,25.895168,-80.123150
76,PAST SALE,2024-11-18,Condo/Co-op,801 Brickell Key Blvd #2608,Miami,FL,33131.0,1015000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/801-Brickell-K...,MARMLS,A11620231,N,Y,25.768423,-80.186698
25,PAST SALE,2024-11-18,Condo/Co-op,16150 NE 21st Ave #6,North Miami Beach,FL,33162.0,165000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/16...,MARMLS,A11666912,N,Y,25.924646,-80.158184
1,PAST SALE,2024-11-15,Condo/Co-op,540 West Ave #1713,Miami Beach,FL,33139.0,793000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/540-West...,MARMLS,A11693360,N,Y,25.775240,-80.141945
57,PAST SALE,2024-11-15,Condo/Co-op,3193 NW 11 St Unit 6-E,Miami,FL,33125.0,445000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3193-NW-11th-S...,MARMLS,A11640759,N,Y,25.784012,-80.247189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,PAST SALE,2024-11-12,Condo/Co-op,650 NE 32nd St #2601,Miami,FL,33137.0,1400000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/650-NE-32nd-St...,MARMLS,A11616420,N,Y,25.806902,-80.186249
64,PAST SALE,2024-11-12,Condo/Co-op,9009 NE 4th Ave Rd #9009,Miami Shores,FL,33138.0,265000.0,2.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Shores/9009-NE...,MARMLS,A11632450,N,Y,25.858214,-80.186251
90,PAST SALE,2024-11-11,Condo/Co-op,763 Pennsylvania Ave #116,Miami Beach,FL,33139.0,135000.0,0.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/763-Penn...,MARMLS,A11562923,N,Y,25.778229,-80.133794
75,PAST SALE,2024-11-11,Condo/Co-op,700 NE 25th St #2104,Miami,FL,33137.0,950000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-25th-St...,MARMLS,A11615989,N,Y,25.800829,-80.185785


In [10]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [11]:
df_top_ten = df.head(10)

## Collect Agent Information

In [12]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [13]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [14]:
response_df = pd.DataFrame(response_list)

In [15]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [16]:
df_top_ten = merged_df

## Current Week's Values

In [17]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

102


In [18]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$1,022,796


In [19]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$606


In [20]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$104,325,203


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [21]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Miami-Dade
Input Previous Week Condo Sales Total (number units sold):
83
Input Previous Week Condo Average Sales Price:
735595
Input Previous Week Condo Average PSF:
557
Input Previous Week Condo Sales total (ex: 198_000_000)
92_500_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [22]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [23]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [24]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [25]:
pd.set_option('display.max_columns',None)

In [26]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-15,Condo/Co-op,5211 Fisher Island Dr #5211,Miami Beach,FL,33109.0,11350000.0,4.0,5.5,BAYVIEW FISHER ISL CONDO,4010.0,NaN,1992.0,NaN,2830.0,5063.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5211-Fis...,MARMLS,A11500565,N,Y,25.762689,-80.148415,https://www.redfin.com/FL/Miami-Beach/5211-Fis...,Julian Johnston,The Corcoran Group,Richard Taylor,The Corcoran Group,None,None,None,None,1,orange


In [27]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [28]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-15,Condo/Co-op,5211 Fisher Island Dr #5211,Miami Beach,FL,33109.0,11350000.0,4.0,5.5,BAYVIEW FISHER ISL CONDO,4010.0,NaN,1992.0,NaN,2830.0,5063.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5211-Fis...,MARMLS,A11500565,N,Y,25.762689,-80.148415,https://www.redfin.com/FL/Miami-Beach/5211-Fis...,Julian Johnston,The Corcoran Group,Richard Taylor,The Corcoran Group,None,None,None,None,1,orange
1,PAST SALE,2024-11-14,Condo/Co-op,9701 Collins Ave Unit 701S,Bal Harbour,FL,33154.0,8300000.0,3.0,3.5,BAL HARBOUR NORTH SOUTH C,3884.0,NaN,2011.0,NaN,2137.0,9097.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9701-Col...,MARMLS,A11658031,N,Y,25.888491,-80.122881,https://www.redfin.com/FL/Bal-Harbour/9701-Col...,Ryan Mendell,Maxwell E Realty Inc,None,None,Marina Zaslavskaya,Keller Williams Elite Properties,None,None,2,blue
2,PAST SALE,2024-11-12,Condo/Co-op,9701 Collins Ave Unit 1602S,Bal Harbour,FL,33154.0,7100000.0,3.0,3.5,BAL HARBOUR NORTH SOUTH C,3424.0,NaN,2012.0,NaN,2074.0,8074.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9701-Col...,MARMLS,A11587733,N,Y,25.888491,-80.122881,https://www.redfin.com/FL/Bal-Harbour/9701-Col...,Roberta Ingletto,RGI Realty,Giovanna Ingletto Amaral,RGI Realty,Cristiano Piquet,Piquet Realty,None,None,3,blue
3,PAST SALE,2024-11-18,Condo/Co-op,10201 Collins Ave #2406,Bal Harbour,FL,33154.0,6562500.0,2.0,3.5,OCEANA BAL HARBOUR CONDO,2625.0,NaN,2016.0,NaN,2500.0,5360.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,MARMLS,A11559220,N,Y,25.895168,-80.123150,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,Chad Carroll,"Compass Florida, LLC",None,None,Chad Carroll,"Compass Florida, LLC",Chad Carroll,"Compass Florida, LLC",4,blue
4,PAST SALE,2024-11-13,Condo/Co-op,2675 S Bayshore Dr Unit 702S,Miami,FL,33133.0,5900000.0,5.0,5.5,DINNER KEY,3929.0,NaN,2016.0,NaN,1502.0,12410.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2675-S-Bayshor...,MARMLS,A11510156,N,Y,25.729893,-80.236831,https://www.redfin.com/FL/Miami/2675-S-Bayshor...,Saddy Delgado,One Sotheby's International Realty,Valeria Kelly,One Sotheby's International Realty,Valaree Byrne,BHHS EWM Realty,None,None,5,blue
5,PAST SALE,2024-11-12,Condo/Co-op,15701 Collins Ave #2704,Sunny Isles Beach,FL,33160.0,3500000.0,2.0,2.5,15701 COLLINS CONDO,1571.0,NaN,2020.0,NaN,2228.0,2675.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11498778,N,Y,25.921944,-80.121716,https://www.redfin.com/FL/North-Miami-Beach/15...,Nelly Estopinan,"Compass Florida, LLC",None,None,Nancy Batchelor,"Compass Florida, LLC.",Michelle Shurtleff,"Compass Florida, LLC.",6,blue
6,PAST SALE,2024-11-12,Condo/Co-op,2800 Island Blvd #2901,Aventura,FL,33160.0,2712500.0,4.0,6.5,2800 ISL BLVD WILLIAMS IS,4220.0,NaN,1990.0,NaN,643.0,6576.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/2800-Island...,MARMLS,A11545415,N,Y,25.939732,-80.134311,https://www.redfin.com/FL/Aventura/2800-Island...,Joel Matus,"Williams Island Realty, LLC",None,None,Joel Matus,"Williams Island Realty, LLC",None,None,7,blue
7,PAST SALE,2024-11-14,Condo/Co-op,18975 Collins Ave #3805,Sunny Isles Beach,FL,33160.0,2500000.0,2.0,2.0,18975 COLLINS CONDO,1233.0,NaN,2019.0,NaN,2028.0,2027.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11620144,N,Y,25.951740,-80.119755,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Jolein Mustafa,London Foster Realty,None,None,Ashton Coleman,One Sotheby's International Re,None,None,8,blue
8,PAST SALE,2024-11-15,Condo/Co-op,3201 NE 183rd St #2507,Aventura,FL

In [29]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'Nov. 3rd — Nov. 10th')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [30]:
m.save('index.html')

## Data snagger

In [31]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [32]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-15,Condo/Co-op,5211 Fisher Island Dr #5211,Miami Beach,FL,33109.0,11350000.0,4.0,5.5,BAYVIEW FISHER ISL CONDO,4010.0,NaN,1992.0,NaN,2830.0,5063.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5211-Fis...,MARMLS,A11500565,N,Y,25.762689,-80.148415,https://www.redfin.com/FL/Miami-Beach/5211-Fis...,Julian Johnston,The Corcoran Group,Richard Taylor,The Corcoran Group,None,None,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [33]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_11182024


# CREATE TEMPLATE 

In [34]:
muni_set = set(df_top_ten['CITY'])

In [35]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [36]:
df_top_ten.reset_index(inplace=True,drop=True)

In [37]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [38]:
top_sale

'BAYVIEW FISHER ISL CONDO at 5211 Fisher Island Dr #5211 in Miami Beach'

In [39]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [40]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [41]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-15,Condo/Co-op,5211 Fisher Island Dr #5211,Miami Beach,FL,33109.0,11350000.0,4.0,5.5,BAYVIEW FISHER ISL CONDO,4010.0,NaN,1992.0,NaN,2830.0,5063.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5211-Fis...,MARMLS,A11500565,N,Y,25.762689,-80.148415,https://www.redfin.com/FL/Miami-Beach/5211-Fis...,Julian Johnston,The Corcoran Group,Richard Taylor,The Corcoran Group,None,None,None,None,1,orange,BAYVIEW FISHER ISL CONDO at 5211 Fisher Island...
1,PAST SALE,2024-11-14,Condo/Co-op,9701 Collins Ave Unit 701S,Bal Harbour,FL,33154.0,8300000.0,3.0,3.5,BAL HARBOUR NORTH SOUTH C,3884.0,NaN,2011.0,NaN,2137.0,9097.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9701-Col...,MARMLS,A11658031,N,Y,25.888491,-80.122881,https://www.redfin.com/FL/Bal-Harbour/9701-Col...,Ryan Mendell,Maxwell E Realty Inc,None,None,Marina Zaslavskaya,Keller Williams Elite Properties,None,None,2,blue,BAL HARBOUR NORTH SOUTH C at 9701 Collins Ave ...
2,PAST SALE,2024-11-12,Condo/Co-op,9701 Collins Ave Unit 1602S,Bal Harbour,FL,33154.0,7100000.0,3.0,3.5,BAL HARBOUR NORTH SOUTH C,3424.0,NaN,2012.0,NaN,2074.0,8074.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9701-Col...,MARMLS,A11587733,N,Y,25.888491,-80.122881,https://www.redfin.com/FL/Bal-Harbour/9701-Col...,Roberta Ingletto,RGI Realty,Giovanna Ingletto Amaral,RGI Realty,Cristiano Piquet,Piquet Realty,None,None,3,blue,BAL HARBOUR NORTH SOUTH C at 9701 Collins Ave ...
3,PAST SALE,2024-11-18,Condo/Co-op,10201 Collins Ave #2406,Bal Harbour,FL,33154.0,6562500.0,2.0,3.5,OCEANA BAL HARBOUR CONDO,2625.0,NaN,2016.0,NaN,2500.0,5360.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,MARMLS,A11559220,N,Y,25.895168,-80.123150,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,Chad Carroll,"Compass Florida, LLC",None,None,Chad Carroll,"Compass Florida, LLC",Chad Carroll,"Compass Florida, LLC",4,blue,OCEANA BAL HARBOUR CONDO at 10201 Collins Ave ...
4,PAST SALE,2024-11-13,Condo/Co-op,2675 S Bayshore Dr Unit 702S,Miami,FL,33133.0,5900000.0,5.0,5.5,DINNER KEY,3929.0,NaN,2016.0,NaN,1502.0,12410.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2675-S-Bayshor...,MARMLS,A11510156,N,Y,25.729893,-80.236831,https://www.redfin.com/FL/Miami/2675-S-Bayshor...,Saddy Delgado,One Sotheby's International Realty,Valeria Kelly,One Sotheby's International Realty,Valaree Byrne,BHHS EWM Realty,None,None,5,blue,DINNER KEY at 2675 S Bayshore Dr Unit 702S in ...
5,PAST SALE,2024-11-12,Condo/Co-op,15701 Collins Ave #2704,Sunny Isles Beach,FL,33160.0,3500000.0,2.0,2.5,15701 COLLINS CONDO,1571.0,NaN,2020.0,NaN,2228.0,2675.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11498778,N,Y,25.921944,-80.121716,https://www.redfin.com/FL/North-Miami-Beach/15...,Nelly Estopinan,"Compass Florida, LLC",None,None,Nancy Batchelor,"Compass Florida, LLC.",Michelle Shurtleff,"Compass Florida, LLC.",6,blue,15701 COLLINS CONDO at 15701 Collins Ave #2704...
6,PAST SALE,2024-11-12,Condo/Co-op,2800 Island Blvd #2901,Aventura,FL,33160.0,2712500.0,4.0,6.5,2800 ISL BLVD WILLIAMS IS,4220.0,NaN,1990.0,NaN,643.0,6576.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/2800-Island...,MARMLS,A11545415,N,Y,25.939732,-80.134311,https://www.redfin.com/FL/Aventura/2800-Island...,Joel Matus,"Williams Island Realty, LLC",None,None,Joel Matus,"Williams Island Realty, LLC",None,None,7,blue,2800 ISL BLVD WILLIAMS IS at 2800 Island Blvd ...
7,PAST SALE,2024-11-14,Condo/Co-op,18975 Collins Ave #3805,Sunny Isles Beach,FL,33160.0,2500000.0,2.0,2.0,18975 COLLINS CONDO,1233.0,Na

In [42]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: BAYVIEW FISHER ISL CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,350,000M to $11,350,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Miami-Dade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Bal Harbour, Aventura, Miami Beach, Sunny Isles Beach, Miami

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 102 condo sales totaling $104,325,203 million from Nov. 10th to Nov. 17th. The previous week, brokers closed 83 condo sales totaling $92,500,000.

Last week’s units sold for an average of $1,022,796, higher than the $735,595 average price from the previous week. The a

In [43]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [44]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [45]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [46]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-15,Condo/Co-op,5211 Fisher Island Dr #5211,Miami Beach,FL,33109.0,11350000.0,4.0,5.5,BAYVIEW FISHER ISL CONDO,4010.0,NaN,1992.0,NaN,2830.0,5063.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5211-Fis...,MARMLS,A11500565,N,Y,25.762689,-80.148415,https://www.redfin.com/FL/Miami-Beach/5211-Fis...,Julian Johnston,The Corcoran Group,Richard Taylor,The Corcoran Group,None,None,None,None,1,orange,BAYVIEW FISHER ISL CONDO at 5211 Fisher Island...
1,PAST SALE,2024-11-14,Condo/Co-op,9701 Collins Ave Unit 701S,Bal Harbour,FL,33154.0,8300000.0,3.0,3.5,BAL HARBOUR NORTH SOUTH C,3884.0,NaN,2011.0,NaN,2137.0,9097.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9701-Col...,MARMLS,A11658031,N,Y,25.888491,-80.122881,https://www.redfin.com/FL/Bal-Harbour/9701-Col...,Ryan Mendell,Maxwell E Realty Inc,None,None,Marina Zaslavskaya,Keller Williams Elite Properties,None,None,2,blue,BAL HARBOUR NORTH SOUTH C at 9701 Collins Ave ...
2,PAST SALE,2024-11-12,Condo/Co-op,9701 Collins Ave Unit 1602S,Bal Harbour,FL,33154.0,7100000.0,3.0,3.5,BAL HARBOUR NORTH SOUTH C,3424.0,NaN,2012.0,NaN,2074.0,8074.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9701-Col...,MARMLS,A11587733,N,Y,25.888491,-80.122881,https://www.redfin.com/FL/Bal-Harbour/9701-Col...,Roberta Ingletto,RGI Realty,Giovanna Ingletto Amaral,RGI Realty,Cristiano Piquet,Piquet Realty,None,None,3,blue,BAL HARBOUR NORTH SOUTH C at 9701 Collins Ave ...
3,PAST SALE,2024-11-18,Condo/Co-op,10201 Collins Ave #2406,Bal Harbour,FL,33154.0,6562500.0,2.0,3.5,OCEANA BAL HARBOUR CONDO,2625.0,NaN,2016.0,NaN,2500.0,5360.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,MARMLS,A11559220,N,Y,25.895168,-80.123150,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,Chad Carroll,"Compass Florida, LLC",None,None,Chad Carroll,"Compass Florida, LLC",Chad Carroll,"Compass Florida, LLC",4,blue,OCEANA BAL HARBOUR CONDO at 10201 Collins Ave ...
4,PAST SALE,2024-11-13,Condo/Co-op,2675 S Bayshore Dr Unit 702S,Miami,FL,33133.0,5900000.0,5.0,5.5,DINNER KEY,3929.0,NaN,2016.0,NaN,1502.0,12410.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2675-S-Bayshor...,MARMLS,A11510156,N,Y,25.729893,-80.236831,https://www.redfin.com/FL/Miami/2675-S-Bayshor...,Saddy Delgado,One Sotheby's International Realty,Valeria Kelly,One Sotheby's International Realty,Valaree Byrne,BHHS EWM Realty,None,None,5,blue,DINNER KEY at 2675 S Bayshore Dr Unit 702S in ...
5,PAST SALE,2024-11-12,Condo/Co-op,15701 Collins Ave #2704,Sunny Isles Beach,FL,33160.0,3500000.0,2.0,2.5,15701 COLLINS CONDO,1571.0,NaN,2020.0,NaN,2228.0,2675.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11498778,N,Y,25.921944,-80.121716,https://www.redfin.com/FL/North-Miami-Beach/15...,Nelly Estopinan,"Compass Florida, LLC",None,None,Nancy Batchelor,"Compass Florida, LLC.",Michelle Shurtleff,"Compass Florida, LLC.",6,blue,15701 COLLINS CONDO at 15701 Collins Ave #2704...
6,PAST SALE,2024-11-12,Condo/Co-op,2800 Island Blvd #2901,Aventura,FL,33160.0,2712500.0,4.0,6.5,2800 ISL BLVD WILLIAMS IS,4220.0,NaN,1990.0,NaN,643.0,6576.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/2800-Island...,MARMLS,A11545415,N,Y,25.939732,-80.134311,https://www.redfin.com/FL/Aventura/2800-Island...,Joel Matus,"Williams Island Realty, LLC",None,None,Joel Matus,"Williams Island Realty, LLC",None,None,7,blue,2800 ISL BLVD WILLIAMS IS at 2800 Island Blvd ...
7,PAST SALE,2024-11-14,Condo/Co-op,18975 Collins Ave #3805,Sunny Isles Beach,FL,33160.0,2500000.0,2.0,2.0,18975 COLLINS CONDO,1233.0,Na

In [47]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: BAYVIEW FISHER ISL CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,350,000M to $11,350,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Miami-Dade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Bal Harbour, Aventura, Miami Beach, Sunny Isles Beach, Miami

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 102 condo sales totaling $104,325,203 million from Nov. 10th to Nov. 17th. The previous week, brokers closed 83 condo sales totaling $92,500,000.

Last week’s units sold for an average of $1,022,796, higher than the $735,595 average price from the previous week. The a

In [48]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Miami-Beach/5211-Fisher-Island-Dr-33109/unit-5211/home/43102905


In [55]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Bal-Harbour/9701-Collins-Ave-33154/unit-701S/home/50121176


In [50]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami/45-SW-9th-St-33130/unit-4806/home/79134870


In [51]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Aventura/3201-NE-183rd-St-33160/unit-2507/home/43312194


In [52]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami-Beach/5211-Fisher-Island-Dr-33109/unit-5211/home/43102905


In [53]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Aventura/2800-Island-Blvd-33160/unit-2901/home/42948421


In [54]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-15,Condo/Co-op,5211 Fisher Island Dr #5211,Miami Beach,FL,33109.0,11350000.0,4.0,5.5,BAYVIEW FISHER ISL CONDO,4010.0,NaN,1992.0,NaN,2830.0,5063.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5211-Fis...,MARMLS,A11500565,N,Y,25.762689,-80.148415,https://www.redfin.com/FL/Miami-Beach/5211-Fis...,Julian Johnston,The Corcoran Group,Richard Taylor,The Corcoran Group,None,None,None,None,1,orange,BAYVIEW FISHER ISL CONDO at 5211 Fisher Island...
1,PAST SALE,2024-11-14,Condo/Co-op,9701 Collins Ave Unit 701S,Bal Harbour,FL,33154.0,8300000.0,3.0,3.5,BAL HARBOUR NORTH SOUTH C,3884.0,NaN,2011.0,NaN,2137.0,9097.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9701-Col...,MARMLS,A11658031,N,Y,25.888491,-80.122881,https://www.redfin.com/FL/Bal-Harbour/9701-Col...,Ryan Mendell,Maxwell E Realty Inc,None,None,Marina Zaslavskaya,Keller Williams Elite Properties,None,None,2,blue,BAL HARBOUR NORTH SOUTH C at 9701 Collins Ave ...
2,PAST SALE,2024-11-12,Condo/Co-op,9701 Collins Ave Unit 1602S,Bal Harbour,FL,33154.0,7100000.0,3.0,3.5,BAL HARBOUR NORTH SOUTH C,3424.0,NaN,2012.0,NaN,2074.0,8074.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9701-Col...,MARMLS,A11587733,N,Y,25.888491,-80.122881,https://www.redfin.com/FL/Bal-Harbour/9701-Col...,Roberta Ingletto,RGI Realty,Giovanna Ingletto Amaral,RGI Realty,Cristiano Piquet,Piquet Realty,None,None,3,blue,BAL HARBOUR NORTH SOUTH C at 9701 Collins Ave ...
3,PAST SALE,2024-11-18,Condo/Co-op,10201 Collins Ave #2406,Bal Harbour,FL,33154.0,6562500.0,2.0,3.5,OCEANA BAL HARBOUR CONDO,2625.0,NaN,2016.0,NaN,2500.0,5360.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,MARMLS,A11559220,N,Y,25.895168,-80.123150,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,Chad Carroll,"Compass Florida, LLC",None,None,Chad Carroll,"Compass Florida, LLC",Chad Carroll,"Compass Florida, LLC",4,blue,OCEANA BAL HARBOUR CONDO at 10201 Collins Ave ...
4,PAST SALE,2024-11-13,Condo/Co-op,2675 S Bayshore Dr Unit 702S,Miami,FL,33133.0,5900000.0,5.0,5.5,DINNER KEY,3929.0,NaN,2016.0,NaN,1502.0,12410.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2675-S-Bayshor...,MARMLS,A11510156,N,Y,25.729893,-80.236831,https://www.redfin.com/FL/Miami/2675-S-Bayshor...,Saddy Delgado,One Sotheby's International Realty,Valeria Kelly,One Sotheby's International Realty,Valaree Byrne,BHHS EWM Realty,None,None,5,blue,DINNER KEY at 2675 S Bayshore Dr Unit 702S in ...
5,PAST SALE,2024-11-12,Condo/Co-op,15701 Collins Ave #2704,Sunny Isles Beach,FL,33160.0,3500000.0,2.0,2.5,15701 COLLINS CONDO,1571.0,NaN,2020.0,NaN,2228.0,2675.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11498778,N,Y,25.921944,-80.121716,https://www.redfin.com/FL/North-Miami-Beach/15...,Nelly Estopinan,"Compass Florida, LLC",None,None,Nancy Batchelor,"Compass Florida, LLC.",Michelle Shurtleff,"Compass Florida, LLC.",6,blue,15701 COLLINS CONDO at 15701 Collins Ave #2704...
6,PAST SALE,2024-11-12,Condo/Co-op,2800 Island Blvd #2901,Aventura,FL,33160.0,2712500.0,4.0,6.5,2800 ISL BLVD WILLIAMS IS,4220.0,NaN,1990.0,NaN,643.0,6576.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/2800-Island...,MARMLS,A11545415,N,Y,25.939732,-80.134311,https://www.redfin.com/FL/Aventura/2800-Island...,Joel Matus,"Williams Island Realty, LLC",None,None,Joel Matus,"Williams Island Realty, LLC",None,None,7,blue,2800 ISL BLVD WILLIAMS IS at 2800 Island Blvd ...
7,PAST SALE,2024-11-14,Condo/Co-op,18975 Collins Ave #3805,Sunny Isles Beach,FL,33160.0,2500000.0,2.0,2.0,18975 COLLINS CONDO,1233.0,Na

## Time on Market Calculator

In [57]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 9, 12) ## List (Earlier) date
date2 = datetime(2024, 11, 14) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

63


## Clean CSV for Datawrapper Chart

In [58]:
chart_df = df_top_ten

In [59]:
chart_df = chart_df.fillna(" ")

In [60]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [61]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [62]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [63]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [64]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [65]:
csv_date_string = today.strftime("%m_%d_%Y")

In [66]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [67]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2024-11-15,5211 Fisher Island Dr #5211 in Miami Beach,"$11,350,000",4,5.5,"4,010",1992,"$2,830",MARMLS,A11500565,Julian Johnston The Corcoran Group Richard Tay...,
1,2024-11-14,9701 Collins Ave Unit 701S in Bal Harbour,"$8,300,000",3,3.5,"3,884",2011,"$2,137",MARMLS,A11658031,Ryan Mendell Maxwell E Realty Inc,Marina Zaslavskaya Keller Williams Elite Prope...
2,2024-11-12,9701 Collins Ave Unit 1602S in Bal Harbour,"$7,100,000",3,3.5,"3,424",2012,"$2,074",MARMLS,A11587733,Roberta Ingletto RGI Realty Giovanna Ingletto ...,Cristiano Piquet Piquet Realty
3,2024-11-18,10201 Collins Ave #2406 in Bal Harbour,"$6,562,500",2,3.5,"2,625",2016,"$2,500",MARMLS,A11559220,"Chad Carroll Compass Florida, LLC","Chad Carroll Compass Florida, LLC Chad Carroll..."
4,2024-11-13,2675 S Bayshore Dr Unit 702S in Miami,"$5,900,000",5,5.5,"3,929",2016,"$1,502",MARMLS,A11510156,Saddy Delgado One Sotheby's International Real...,Valaree Byrne BHHS EWM Realty
5,2024-11-12,15701 Collins Ave #2704 in Sunny Isles Beach,"$3,500,000",2,2.5,"1,571",2020,"$2,228",MARMLS,A11498778,"Nelly Estopinan Compass Florida, LLC","Nancy Batchelor Compass Florida, LLC. Michelle..."
6,2024-11-12,2800 Island Blvd #2901 in Aventura,"$2,712,500",4,6.5,"4,220",1990,$643,MARMLS,A11545415,"Joel Matus Williams Island Realty, LLC","Joel Matus Williams Island Realty, LLC"
7,2024-11-14,18975 Collins Ave #3805 in Sunny Isles Beach,"$2,500,000",2,2.0,"1,233",2019,"$2,028",MARMLS,A11620144,Jolein Mustafa London Foster Realty,Ashton Coleman One Sotheby's International Re
8,2024-11-15,3201 NE 183rd St #2507 in Aventura,"$2,466,203",3,3.0,"3,274",2003,$753,MARMLS,A11690712,"Gail Suzyn Compass Florida, LLC. Karen Matluck...",Sigal Dagan One Sotheby's International Realty...
9,2024-11-12,45 SW 9th St Ph 4806 in Miami,"$2,350,000",4,4.5,"2,477",2017,$949,MARMLS,A11529862,Maruja Gil Keller Williams Capital Realty,Yeanli Velez LPT Realty
